In [117]:
pip install furl

You should consider upgrading via the '/Users/mingtayang/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [254]:
import requests
import bs4
import urllib.parse
import time
import random
import re
from furl import furl

# Define Variables

In [268]:
# Define keywords
keyword_primary = "iOS"
keyword_secondary_list = ['遠端', '遠距', '在家']

In [269]:
# Define original url
url_original = 'https://www.104.com.tw/jobs/search/?ro=0&jobcat=2007001004&kwop=7&keyword=%E9%81%A0%E7%AB%AF&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&order=14&asc=0&page=1&mode=s&jobsource=104_bank1'

# Gather web page results

In [271]:
# Define how to tell a page is invalid
def is_invalid_page(content):
    select = content.find('p', class_='b-tit')
    return True if select else False

In [272]:
f = furl(url_original)

In [273]:
f.args['keyword'] = keyword_primary 

In [274]:
page_soups = []
page_count = 1
while True:
    print(f'Start fetching page {page_count}...')
    f.args['page'] = page_count
    print(f.url)
    
    r = requests.get(f.url) 
    if r.status_code != 200:
        break
    
    soup = bs4.BeautifulSoup(r.text, 'lxml')
    if is_invalid_page(soup):
        break
        
    page_soups.append(soup)
    time.sleep(random.randint(3, 5))
    page_count += 1

Start fetching page 1...
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2007001004&kwop=7&keyword=iOS&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&order=14&asc=0&page=1&mode=s&jobsource=104_bank1
Start fetching page 2...
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2007001004&kwop=7&keyword=iOS&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&order=14&asc=0&page=2&mode=s&jobsource=104_bank1
Start fetching page 3...
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2007001004&kwop=7&keyword=iOS&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&order=14&asc=0&page=3&mode=s&jobsource=104_bank1
Start fetching page 4...
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2007001004&kwop=7&keyword=iOS&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&order=14&asc=0&page=4&mode=s&jobsource=104_bank1
Start fetching page 5...
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2007001004&kwop=7&keyword=iOS&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&order=14&asc=0&page=5&mode=s&jobsource=

Start fetching page 42...
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2007001004&kwop=7&keyword=iOS&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&order=14&asc=0&page=42&mode=s&jobsource=104_bank1
Start fetching page 43...
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2007001004&kwop=7&keyword=iOS&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&order=14&asc=0&page=43&mode=s&jobsource=104_bank1
Start fetching page 44...
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2007001004&kwop=7&keyword=iOS&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&order=14&asc=0&page=44&mode=s&jobsource=104_bank1
Start fetching page 45...
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2007001004&kwop=7&keyword=iOS&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&order=14&asc=0&page=45&mode=s&jobsource=104_bank1
Start fetching page 46...
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2007001004&kwop=7&keyword=iOS&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&order=14&asc=0&page=46&mode=s&

In [275]:
print(f'Found {len(page_soups)} Pages.')

Found 63 Pages.


# Use secondary keywords to filter the results

In [276]:
regex = '|'.join(keyword_secondary_list)
flags = re.IGNORECASE

In [277]:
results = []
job_urls = []
for soup in page_soups:
    articles = soup.find_all('article')
    job_urls.extend([urllib.parse.urljoin(url, article.find('a')['href']) for article in articles if article.find('a')])
print(f'Totally {len(job_urls)} jobs to filter.')
    
count = 0
for url in job_urls:
    r = requests.get(url)
    print(f'Start fetching url: {url}...')

    if re.search(regex, r.text.replace('ios-app', 'app'), flags = re.IGNORECASE):
        results.append(url)

    count += 1
    print(f'Filter progress: {round(count/len(job_urls)*100, 2)}%')

    time.sleep(random.randint(3, 5))


Totally 1246 jobs to filter.
Start fetching url: https://www.104.com.tw/job/6ztl0?jobsource=hotjob_chr...
Filter progress: 0.08%
Start fetching url: https://www.104.com.tw/job/73nqo?jobsource=hotjob_chr...
Filter progress: 0.16%
Start fetching url: https://www.104.com.tw/job/73ppk?jobsource=jolist_c_relevance...
Filter progress: 0.24%
Start fetching url: https://www.104.com.tw/job/79pg9?jobsource=jolist_c_relevance...
Filter progress: 0.32%
Start fetching url: https://www.104.com.tw/job/78ldl?jobsource=jolist_c_relevance...
Filter progress: 0.4%
Start fetching url: https://www.104.com.tw/job/79z37?jobsource=jolist_c_relevance...
Filter progress: 0.48%
Start fetching url: https://www.104.com.tw/job/76tqv?jobsource=jolist_c_relevance...
Filter progress: 0.56%
Start fetching url: https://www.104.com.tw/job/788vo?jobsource=jolist_c_relevance...
Filter progress: 0.64%
Start fetching url: https://www.104.com.tw/job/72rbn?jobsource=jolist_c_relevance...
Filter progress: 0.72%
Start fetching u

Start fetching url: https://www.104.com.tw/job/4ij8e?jobsource=jolist_b_relevance...
Filter progress: 6.18%
Start fetching url: https://www.104.com.tw/job/4sgob?jobsource=jolist_b_relevance...
Filter progress: 6.26%
Start fetching url: https://www.104.com.tw/job/6psuu?jobsource=jolist_b_relevance...
Filter progress: 6.34%
Start fetching url: https://www.104.com.tw/job/73v1u?jobsource=jolist_b_relevance...
Filter progress: 6.42%
Start fetching url: https://www.104.com.tw/job/6dot6?jobsource=jolist_b_relevance...
Filter progress: 6.5%
Start fetching url: https://www.104.com.tw/job/79tui?jobsource=jolist_b_relevance...
Filter progress: 6.58%
Start fetching url: https://www.104.com.tw/job/5e5di?jobsource=jolist_b_relevance...
Filter progress: 6.66%
Start fetching url: https://www.104.com.tw/job/75g17?jobsource=jolist_b_relevance...
Filter progress: 6.74%
Start fetching url: https://www.104.com.tw/job/6z8kw?jobsource=jolist_b_relevance...
Filter progress: 6.82%
Start fetching url: https://w

Start fetching url: https://www.104.com.tw/job/65s6o?jobsource=104_bank1...
Filter progress: 12.36%
Start fetching url: https://www.104.com.tw/job/4g5c7?jobsource=104_bank1...
Filter progress: 12.44%
Start fetching url: https://www.104.com.tw/job/6zmzq?jobsource=104_bank1...
Filter progress: 12.52%
Start fetching url: https://www.104.com.tw/job/790tc?jobsource=104_bank1...
Filter progress: 12.6%
Start fetching url: https://www.104.com.tw/job/70a6e?jobsource=104_bank1...
Filter progress: 12.68%
Start fetching url: https://www.104.com.tw/job/73ig2?jobsource=104_bank1...
Filter progress: 12.76%
Start fetching url: https://www.104.com.tw/job/6fq53?jobsource=104_bank1...
Filter progress: 12.84%
Start fetching url: https://www.104.com.tw/job/737t5?jobsource=104_bank1...
Filter progress: 12.92%
Start fetching url: https://www.104.com.tw/job/31qjp?jobsource=104_bank1...
Filter progress: 13.0%
Start fetching url: https://www.104.com.tw/job/6snkh?jobsource=jolist_a_relevance...
Filter progress: 

Start fetching url: https://www.104.com.tw/job/6o05x?jobsource=jolist_c_relevance...
Filter progress: 18.46%
Start fetching url: https://www.104.com.tw/job/6tx53?jobsource=jolist_c_relevance...
Filter progress: 18.54%
Start fetching url: https://www.104.com.tw/job/79ba2?jobsource=jolist_c_relevance...
Filter progress: 18.62%
Start fetching url: https://www.104.com.tw/job/71vg4?jobsource=jolist_c_relevance...
Filter progress: 18.7%
Start fetching url: https://www.104.com.tw/job/75n05?jobsource=jolist_c_relevance...
Filter progress: 18.78%
Start fetching url: https://www.104.com.tw/job/62ih4?jobsource=jolist_c_relevance...
Filter progress: 18.86%
Start fetching url: https://www.104.com.tw/job/5gr55?jobsource=jolist_c_relevance...
Filter progress: 18.94%
Start fetching url: https://www.104.com.tw/job/73kfq?jobsource=jolist_c_relevance...
Filter progress: 19.02%
Start fetching url: https://www.104.com.tw/job/75wbj?jobsource=jolist_c_relevance...
Filter progress: 19.1%
Start fetching url: h

Start fetching url: https://www.104.com.tw/job/6m8u0?jobsource=jolist_c_relevance...
Filter progress: 24.64%
Start fetching url: https://www.104.com.tw/job/79tfj?jobsource=jolist_c_relevance...
Filter progress: 24.72%
Start fetching url: https://www.104.com.tw/job/6d0a3?jobsource=jolist_c_relevance...
Filter progress: 24.8%
Start fetching url: https://www.104.com.tw/job/6thxc?jobsource=jolist_c_relevance...
Filter progress: 24.88%
Start fetching url: https://www.104.com.tw/job/78z0z?jobsource=jolist_c_relevance...
Filter progress: 24.96%
Start fetching url: https://www.104.com.tw/job/6zuaa?jobsource=jolist_c_relevance...
Filter progress: 25.04%
Start fetching url: https://www.104.com.tw/job/72fb1?jobsource=jolist_c_relevance...
Filter progress: 25.12%
Start fetching url: https://www.104.com.tw/job/72uuq?jobsource=jolist_c_relevance...
Filter progress: 25.2%
Start fetching url: https://www.104.com.tw/job/6vab0?jobsource=jolist_c_relevance...
Filter progress: 25.28%
Start fetching url: h

...

Start fetching url: https://www.104.com.tw/job/78nls?jobsource=jolist_a_relevance...
Filter progress: 74.16%
Start fetching url: https://www.104.com.tw/job/73w7a?jobsource=jolist_a_relevance...
Filter progress: 74.24%
Start fetching url: https://www.104.com.tw/job/6unk8?jobsource=jolist_a_relevance...
Filter progress: 74.32%
Start fetching url: https://www.104.com.tw/job/3c1us?jobsource=jolist_a_relevance...
Filter progress: 74.4%
Start fetching url: https://www.104.com.tw/job/68xij?jobsource=jolist_a_relevance...
Filter progress: 74.48%
Start fetching url: https://www.104.com.tw/job/77keg?jobsource=jolist_a_relevance...
Filter progress: 74.56%
Start fetching url: https://www.104.com.tw/job/747nj?jobsource=jolist_a_relevance...
Filter progress: 74.64%
Start fetching url: https://www.104.com.tw/job/784d3?jobsource=jolist_a_relevance...
Filter progress: 74.72%
Start fetching url: https://www.104.com.tw/job/4t096?jobsource=jolist_a_relevance...
Filter progress: 74.8%
Start fetching url: h

Start fetching url: https://www.104.com.tw/job/77kio?jobsource=104_bank1...
Filter progress: 80.34%
Start fetching url: https://www.104.com.tw/job/77hht?jobsource=104_bank1...
Filter progress: 80.42%
Start fetching url: https://www.104.com.tw/job/74mtg?jobsource=104_bank1...
Filter progress: 80.5%
Start fetching url: https://www.104.com.tw/job/70cy9?jobsource=104_bank1...
Filter progress: 80.58%
Start fetching url: https://www.104.com.tw/job/3x4nc?jobsource=104_bank1...
Filter progress: 80.66%
Start fetching url: https://www.104.com.tw/job/4atiw?jobsource=104_bank1...
Filter progress: 80.74%
Start fetching url: https://www.104.com.tw/job/6xy5g?jobsource=104_bank1...
Filter progress: 80.82%
Start fetching url: https://www.104.com.tw/job/5z0q6?jobsource=104_bank1...
Filter progress: 80.9%
Start fetching url: https://www.104.com.tw/job/76fw3?jobsource=104_bank1...
Filter progress: 80.98%
Start fetching url: https://www.104.com.tw/job/3gmoz?jobsource=104_bank1...
Filter progress: 81.06%
St

Start fetching url: https://www.104.com.tw/job/6v5y6?jobsource=jolist_a_relevance...
Filter progress: 86.6%
Start fetching url: https://www.104.com.tw/job/6ky6h?jobsource=jolist_a_relevance...
Filter progress: 86.68%
Start fetching url: https://www.104.com.tw/job/6sdh7?jobsource=jolist_a_relevance...
Filter progress: 86.76%
Start fetching url: https://www.104.com.tw/job/6zpwa?jobsource=jolist_a_relevance...
Filter progress: 86.84%
Start fetching url: https://www.104.com.tw/job/5dcxa?jobsource=104_bank1...
Filter progress: 86.92%
Start fetching url: https://www.104.com.tw/job/787y6?jobsource=104_bank1...
Filter progress: 87.0%
Start fetching url: https://www.104.com.tw/job/77nth?jobsource=104_bank1...
Filter progress: 87.08%
Start fetching url: https://www.104.com.tw/job/74sd0?jobsource=104_bank1...
Filter progress: 87.16%
Start fetching url: https://www.104.com.tw/job/2s2ak?jobsource=104_bank1...
Filter progress: 87.24%
Start fetching url: https://www.104.com.tw/job/6v04u?jobsource=104

Start fetching url: https://www.104.com.tw/job/70swk?jobsource=104_bank1...
Filter progress: 93.18%
Start fetching url: https://www.104.com.tw/job/75bmd?jobsource=104_bank1...
Filter progress: 93.26%
Start fetching url: https://www.104.com.tw/job/6lzj7?jobsource=jolist_b_relevance...
Filter progress: 93.34%
Start fetching url: https://www.104.com.tw/job/75bmd?jobsource=jolist_b_relevance...
Filter progress: 93.42%
Start fetching url: https://www.104.com.tw/job/75bmg?jobsource=jolist_b_relevance...
Filter progress: 93.5%
Start fetching url: https://www.104.com.tw/job/75bnv?jobsource=jolist_b_relevance...
Filter progress: 93.58%
Start fetching url: https://www.104.com.tw/job/6y1zf?jobsource=jolist_b_relevance...
Filter progress: 93.66%
Start fetching url: https://www.104.com.tw/job/6vt01?jobsource=jolist_b_relevance...
Filter progress: 93.74%
Start fetching url: https://www.104.com.tw/job/74f51?jobsource=jolist_b_relevance...
Filter progress: 93.82%
Start fetching url: https://www.104.co

Start fetching url: https://www.104.com.tw/job/76pzk?jobsource=104_bank1...
Filter progress: 99.52%
Start fetching url: https://www.104.com.tw/job/76pzv?jobsource=104_bank1...
Filter progress: 99.6%
Start fetching url: https://www.104.com.tw/job/74jsg?jobsource=104_bank1...
Filter progress: 99.68%
Start fetching url: https://www.104.com.tw/job/71myw?jobsource=jolist_a_relevance...
Filter progress: 99.76%
Start fetching url: https://www.104.com.tw/job/7640f?jobsource=jolist_a_relevance...
Filter progress: 99.84%
Start fetching url: https://www.104.com.tw/job/4zdk6?jobsource=jolist_a_relevance...
Filter progress: 99.92%
Start fetching url: https://www.104.com.tw/job/6xuu7?jobsource=jolist_a_relevance...
Filter progress: 100.0%


In [280]:
print(f'Found {len(results)} results with the primary keyword "{keyword_primary}" and the secondary keyword "{keyword_secondary_list}".')

Found 11 results with the primary keyword "iOS" and the secondary keyword "['遠端', '遠距', '在家']".


# Show the results

In [279]:
for result in results:
    print(result)

https://www.104.com.tw/job/6tx53?jobsource=jolist_c_relevance
https://www.104.com.tw/job/79dgb?jobsource=jolist_c_relevance
https://www.104.com.tw/job/76qq3?jobsource=104_bank1
https://www.104.com.tw/job/79ymr?jobsource=104_bank1
https://www.104.com.tw/job/79jsb?jobsource=jolist_b_relevance
https://www.104.com.tw/job/77iu9?jobsource=jolist_c_relevance
https://www.104.com.tw/job/6ghzc?jobsource=jolist_c_relevance
https://www.104.com.tw/job/6ow85?jobsource=jolist_c_relevance
https://www.104.com.tw/job/6dkrq?jobsource=jolist_b_relevance
https://www.104.com.tw/job/77b61?jobsource=jolist_b_relevance
https://www.104.com.tw/job/76p93?jobsource=104_bank1
